## Step 50: Interpolate CRWU CREAT Grid Projections


In [17]:
import arcpy,os;

print("Step 50: Interpolate CRWU CREAT Grid Projections");

idw_power = 2;
idw_search_radius_val  = 1;
idw_search_radius_unit = 'degree';

fgdb    = os.getcwd() + os.sep + 'source.gdb';
rezfgdb = os.getcwd() + os.sep + 'results.gdb';
wrkfgdb = os.getcwd() + os.sep + 'working.gdb';

if not arcpy.Exists(rezfgdb):
    print("  Creating new results workspace.");   
    arcpy.CreateFileGDB_management(
         os.path.dirname(rezfgdb)
        ,os.path.basename(rezfgdb)
    );
else:
    print("  Using preexisting results workspace.");

grid      = fgdb + os.sep + 'CRWU_CREAT_Grid_Projections';
grid_cnt  = arcpy.GetCount_management(grid)[0];

print("  Grid Count: " + str(grid_cnt));


Step 50: Interpolate CRWU CREAT Grid Projections
  Using preexisting results workspace.
  Grid Count: 24634


### 50.010: Determine universe of grids with data and buffer extent for interpolation

In [18]:
%%time

in_file   = grid
out_file  = wrkfgdb + os.sep + 'CRWU_CREAT_Grid_Projections_DATA';
out_file2 = wrkfgdb + os.sep + 'CRWU_CREAT_Grid_Projections_CLUMP';
out_file3 = wrkfgdb + os.sep + 'CRWU_CREAT_Grid_Projections_BUFFER';
out_file4 = wrkfgdb + os.sep + 'CRWU_CREAT_Grid_Projections_CENTROIDS';

if arcpy.Exists(out_file):
    arcpy.Delete_management(out_file);
    
if arcpy.Exists(out_file2):
    arcpy.Delete_management(out_file2);
    
if arcpy.Exists(out_file3):
    arcpy.Delete_management(out_file3);
    
if arcpy.Exists(out_file4):
    arcpy.Delete_management(out_file4);

print("  selecting only grids with data");
arcpy.conversion.FeatureClassToFeatureClass(
     in_features  = in_file
    ,out_path     = os.path.dirname(out_file)
    ,out_name     = os.path.basename(out_file)
    ,where_clause = 'PRECIP_2035_HOTDRY_ANNUAL_CHANG IS NOT NULL'
);

grid_data_cnt  = arcpy.GetCount_management(out_file)[0];
print("  Grid Count with Data   : " + str(grid_data_cnt));

print("  generating clumps of grid cells with data");
arcpy.management.Dissolve(
     in_features       = out_file
    ,out_feature_class = out_file2
    ,multi_part        = False
);

grid_clump_cnt  = arcpy.GetCount_management(out_file2)[0];
print("  Clump Count with Data   : " + str(grid_clump_cnt));

if arcpy.Exists('memory/temp'):
    arcpy.Delete_management('memory/temp');
    
arcpy.analysis.Buffer(
     in_features              = out_file2
    ,out_feature_class        = 'memory/temp'
    ,buffer_distance_or_field = '1 degree'
    ,method                   = 'GEODESIC'
);

arcpy.management.Dissolve(
     in_features       = 'memory/temp'
    ,out_feature_class = out_file3
    ,multi_part        = False
);

grid_buffer_cnt  = arcpy.GetCount_management(out_file3)[0];
print("  Buffer Count with Data   : " + str(grid_buffer_cnt));

print("  converting CREAT3 grids with data to centroids");
z = arcpy.management.FeatureToPoint(
     in_features       = out_file
    ,out_feature_class = out_file4
    ,point_location    = 'CENTROID'
);

grid_centroid_cnt  = arcpy.GetCount_management(out_file4)[0];
print("  Centroid Count of Grids with Data: " + str(grid_centroid_cnt));


  selecting only grids with data
  Grid Count with Data   : 4928
  generating clumps of grid cells with data
  Clump Count with Data   : 9
  Buffer Count with Data   : 5
  converting CREAT3 grids with data to centroids
  Centroid Count of Grids with Data: 4928
Wall time: 11.5 s


### 50.020: Determine universe of grids without data and select those within 1 degree buffer for interpolation

In [19]:
%%time

in_file   = grid
in_file2  = wrkfgdb + os.sep + 'CRWU_CREAT_Grid_Projections_BUFFER';
out_file  = wrkfgdb + os.sep + 'CRWU_CREAT_Grid_Projections_NODATA';
out_file2 = wrkfgdb + os.sep + 'CRWU_CREAT_Grid_Projections_INTERP';

if arcpy.Exists(out_file):
    arcpy.Delete_management(out_file);
    
if arcpy.Exists(out_file2):
    arcpy.Delete_management(out_file2);
    
print("  selecting only grids with no data");
arcpy.conversion.FeatureClassToFeatureClass(
     in_features  = in_file
    ,out_path     = os.path.dirname(out_file)
    ,out_name     = os.path.basename(out_file)
    ,where_clause = 'PRECIP_2035_HOTDRY_ANNUAL_CHANG IS NULL'
);

grid_data_cnt  = arcpy.GetCount_management(out_file)[0];
print("  Grid Count with No Data   : " + str(grid_data_cnt));

print("  selecting no data grids within the 1 degree buffers");
fieldmappings = arcpy.FieldMappings();
fieldmappings.addTable(out_file);

arcpy.analysis.SpatialJoin(
     target_features   = out_file
    ,join_features     = in_file2
    ,out_feature_class = out_file2
    ,join_operation    = 'JOIN_ONE_TO_ONE'
    ,join_type         = 'KEEP_COMMON'
    ,field_mapping     = fieldmappings
    ,match_option      = 'WITHIN'
);

arcpy.management.DeleteField(
     in_table   = out_file2
    ,drop_field = [
         'join_count'
        ,'TARGET_FID'
    ]
);

print("  adding row of grids under Puerto Rico for completeness");
flds = [
     'CREAT_ID'
    ,'GRIDCODE'
    ,'HurrGridID'
    ,'grid_over_land'
    ,'Longitude_Centroid'
    ,'Latitude_Centroid'
    ,'Longitude_Min'
    ,'Latitude_Min'
    ,'Longitude_Max'
    ,'Latitude_Max'
    ,'SHAPE@'
];

with arcpy.da.InsertCursor(
    in_table     = out_file2
    ,field_names = flds
) as cursor:
    
    cursor.insertRow((
         None
        ,None
        ,9000001
        ,0
        ,-67.75,17.25
        ,-68   ,17
        ,-67.5 ,17.5
        ,[
            (-68  ,17)
           ,(-68  ,17.5)
           ,(-67.5,17.5)
           ,(-67.5,17)
           ,(-68  ,17)
        ]
    ));
    
    cursor.insertRow((
         None
        ,None
        ,9000002
        ,0
        ,-67.25,17.25
        ,-67.5 ,17
        ,-67   ,17.5
        ,[
            (-67.5,17)
           ,(-67.5,17.5)
           ,(-67  ,17.5)
           ,(-67  ,17)
           ,(-67.5,17)
        ]
    ));
    
    cursor.insertRow((
         None
        ,None
        ,9000003
        ,0
        ,-66.75,17.25
        ,-67   ,17
        ,-66.5   ,17.5
        ,[
            (-67  ,17)
           ,(-67  ,17.5)
           ,(-66.5,17.5)
           ,(-66.5,17)
           ,(-67  ,17)
        ]
    ));
    
    cursor.insertRow((
         None
        ,None
        ,9000004
        ,0
        ,-66.25,17.25
        ,-66.5 ,17
        ,-66   ,17.5
        ,[
            (-66.5,17)
           ,(-66.5,17.5)
           ,(-66  ,17.5)
           ,(-66  ,17)
           ,(-66.5,17)
        ]
    ));
    
    cursor.insertRow((
         None
        ,None
        ,9000005
        ,0
        ,-65.75,17.25
        ,-66   ,17
        ,-65.5 ,17.5
        ,[
            (-66  ,17)
           ,(-66  ,17.5)
           ,(-65.5,17.5)
           ,(-65.5,17)
           ,(-66  ,17)
        ]
    ));
    
    cursor.insertRow((
         None
        ,None
        ,9000006
        ,0
        ,-65.25,17.25
        ,-65.5 ,17
        ,-65   ,17.5
        ,[
            (-65.5,17)
           ,(-65.5,17.5)
           ,(-65  ,17.5)
           ,(-65  ,17)
           ,(-65.5,17)
        ]
    ));

grid_buffer_cnt  = arcpy.GetCount_management(out_file2)[0];
print("  Grid Count with No Data within Buffer  : " + str(grid_buffer_cnt));


  selecting only grids with no data
  Grid Count with No Data   : 19706
  selecting no data grids within the 1 degree buffers
  adding row of grids under Puerto Rico for completeness
  Grid Count with No Data within Buffer  : 1404
Wall time: 10.1 s


### 50.030: Generate interpolation for grids without data within 1 degree

In [20]:
%%time

# set the universal extent slight larger than the input grid extent to account for grids south of Puerto Rico
arcpy.env.extent = arcpy.Extent(
     -177.0,16.0
    ,-64.0 ,72.0
);

in_file   = wrkfgdb + os.sep + 'CRWU_CREAT_Grid_Projections_CENTROIDS';
in_file2  = wrkfgdb + os.sep + 'CRWU_CREAT_Grid_Projections_INTERP'
out_file  = wrkfgdb + os.sep + 'CRWU_CREAT_Grid_Projections_INTERP_WRK'

print("  making working version of interpolated grids")
if arcpy.Exists(out_file):
    arcpy.Delete_management(out_file);

arcpy.conversion.FeatureClassToFeatureClass(
     in_features = in_file2
    ,out_path    = os.path.dirname(out_file)
    ,out_name    = os.path.basename(out_file)
);
    
flds = [
     'TEMP_2035_HOTDRY_JAN_CHANGEF'
    ,'TEMP_2035_HOTDRY_FEB_CHANGEF'
    ,'TEMP_2035_HOTDRY_MAR_CHANGEF'
    ,'TEMP_2035_HOTDRY_APR_CHANGEF'
    ,'TEMP_2035_HOTDRY_MAY_CHANGEF'
    ,'TEMP_2035_HOTDRY_JUN_CHANGEF'
    ,'TEMP_2035_HOTDRY_JUL_CHANGEF'
    ,'TEMP_2035_HOTDRY_AUG_CHANGEF'
    ,'TEMP_2035_HOTDRY_SEP_CHANGEF'
    ,'TEMP_2035_HOTDRY_OCT_CHANGEF'
    ,'TEMP_2035_HOTDRY_NOV_CHANGEF'
    ,'TEMP_2035_HOTDRY_DEC_CHANGEF'
    ,'TEMP_2035_HOTDRY_ANNUAL_CHANGEF'
    
    ,'TEMP_2035_CENTRAL_JAN_CHANGEF'
    ,'TEMP_2035_CENTRAL_FEB_CHANGEF'
    ,'TEMP_2035_CENTRAL_MAR_CHANGEF'
    ,'TEMP_2035_CENTRAL_APR_CHANGEF'
    ,'TEMP_2035_CENTRAL_MAY_CHANGEF'
    ,'TEMP_2035_CENTRAL_JUN_CHANGEF'
    ,'TEMP_2035_CENTRAL_JUL_CHANGEF'
    ,'TEMP_2035_CENTRAL_AUG_CHANGEF'
    ,'TEMP_2035_CENTRAL_SEP_CHANGEF'
    ,'TEMP_2035_CENTRAL_OCT_CHANGEF'
    ,'TEMP_2035_CENTRAL_NOV_CHANGEF'
    ,'TEMP_2035_CENTRAL_DEC_CHANGEF'
    ,'TEMP_2035_CENTRAL_ANNUAL_CHANGE'
    
    ,'TEMP_2035_WETWARM_JAN_CHANGEF'
    ,'TEMP_2035_WETWARM_FEB_CHANGEF'
    ,'TEMP_2035_WETWARM_MAR_CHANGEF'
    ,'TEMP_2035_WETWARM_APR_CHANGEF'
    ,'TEMP_2035_WETWARM_MAY_CHANGEF'
    ,'TEMP_2035_WETWARM_JUN_CHANGEF'
    ,'TEMP_2035_WETWARM_JUL_CHANGEF'
    ,'TEMP_2035_WETWARM_AUG_CHANGEF'
    ,'TEMP_2035_WETWARM_SEP_CHANGEF'
    ,'TEMP_2035_WETWARM_OCT_CHANGEF'
    ,'TEMP_2035_WETWARM_NOV_CHANGEF'
    ,'TEMP_2035_WETWARM_DEC_CHANGEF'
    ,'TEMP_2035_WETWARM_ANNUAL_CHANGE'
    
    ,'TEMP_2060_HOTDRY_JAN_CHANGEF'
    ,'TEMP_2060_HOTDRY_FEB_CHANGEF'
    ,'TEMP_2060_HOTDRY_MAR_CHANGEF'
    ,'TEMP_2060_HOTDRY_APR_CHANGEF'
    ,'TEMP_2060_HOTDRY_MAY_CHANGEF'
    ,'TEMP_2060_HOTDRY_JUN_CHANGEF'
    ,'TEMP_2060_HOTDRY_JUL_CHANGEF'
    ,'TEMP_2060_HOTDRY_AUG_CHANGEF'
    ,'TEMP_2060_HOTDRY_SEP_CHANGEF'
    ,'TEMP_2060_HOTDRY_OCT_CHANGEF'
    ,'TEMP_2060_HOTDRY_NOV_CHANGEF'
    ,'TEMP_2060_HOTDRY_DEC_CHANGEF'
    ,'TEMP_2060_HOTDRY_ANNUAL_CHANGEF'
    
    ,'TEMP_2060_CENTRAL_JAN_CHANGEF'
    ,'TEMP_2060_CENTRAL_FEB_CHANGEF'
    ,'TEMP_2060_CENTRAL_MAR_CHANGEF'
    ,'TEMP_2060_CENTRAL_APR_CHANGEF'
    ,'TEMP_2060_CENTRAL_MAY_CHANGEF'
    ,'TEMP_2060_CENTRAL_JUN_CHANGEF'
    ,'TEMP_2060_CENTRAL_JUL_CHANGEF'
    ,'TEMP_2060_CENTRAL_AUG_CHANGEF'
    ,'TEMP_2060_CENTRAL_SEP_CHANGEF'
    ,'TEMP_2060_CENTRAL_OCT_CHANGEF'
    ,'TEMP_2060_CENTRAL_NOV_CHANGEF'
    ,'TEMP_2060_CENTRAL_DEC_CHANGEF'
    ,'TEMP_2060_CENTRAL_ANNUAL_CHANGE'
    
    ,'TEMP_2060_WETWARM_JAN_CHANGEF'
    ,'TEMP_2060_WETWARM_FEB_CHANGEF'
    ,'TEMP_2060_WETWARM_MAR_CHANGEF'
    ,'TEMP_2060_WETWARM_APR_CHANGEF'
    ,'TEMP_2060_WETWARM_MAY_CHANGEF'
    ,'TEMP_2060_WETWARM_JUN_CHANGEF'
    ,'TEMP_2060_WETWARM_JUL_CHANGEF'
    ,'TEMP_2060_WETWARM_AUG_CHANGEF'
    ,'TEMP_2060_WETWARM_SEP_CHANGEF'
    ,'TEMP_2060_WETWARM_OCT_CHANGEF'
    ,'TEMP_2060_WETWARM_NOV_CHANGEF'
    ,'TEMP_2060_WETWARM_DEC_CHANGEF'
    ,'TEMP_2060_WETWARM_ANNUAL_CHANGE'
    
    ,'PRECIP_2035_HOTDRY_JAN_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_FEB_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_MAR_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_APR_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_MAY_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_JUN_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_JUL_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_AUG_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_SEP_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_OCT_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_NOV_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_DEC_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_ANNUAL_CHANG'

    ,'PRECIP_2035_CENTRAL_JAN_CHANGEP'
    ,'PRECIP_2035_CENTRAL_FEB_CHANGEP'
    ,'PRECIP_2035_CENTRAL_MAR_CHANGEP'
    ,'PRECIP_2035_CENTRAL_APR_CHANGEP'
    ,'PRECIP_2035_CENTRAL_MAY_CHANGEP'
    ,'PRECIP_2035_CENTRAL_JUN_CHANGEP'
    ,'PRECIP_2035_CENTRAL_JUL_CHANGEP'
    ,'PRECIP_2035_CENTRAL_AUG_CHANGEP'
    ,'PRECIP_2035_CENTRAL_SEP_CHANGEP'
    ,'PRECIP_2035_CENTRAL_OCT_CHANGEP'
    ,'PRECIP_2035_CENTRAL_NOV_CHANGEP'
    ,'PRECIP_2035_CENTRAL_DEC_CHANGEP'
    ,'PRECIP_2035_CENTRAL_ANNUAL_CHAN'

    ,'PRECIP_2035_WETWARM_JAN_CHANGEP'
    ,'PRECIP_2035_WETWARM_FEB_CHANGEP'
    ,'PRECIP_2035_WETWARM_MAR_CHANGEP'
    ,'PRECIP_2035_WETWARM_APR_CHANGEP'
    ,'PRECIP_2035_WETWARM_MAY_CHANGEP'
    ,'PRECIP_2035_WETWARM_JUN_CHANGEP'
    ,'PRECIP_2035_WETWARM_JUL_CHANGEP'
    ,'PRECIP_2035_WETWARM_AUG_CHANGEP'
    ,'PRECIP_2035_WETWARM_SEP_CHANGEP'
    ,'PRECIP_2035_WETWARM_OCT_CHANGEP'
    ,'PRECIP_2035_WETWARM_NOV_CHANGEP'
    ,'PRECIP_2035_WETWARM_DEC_CHANGEP'
    ,'PRECIP_2035_WETWARM_ANNUAL_CHAN'

    ,'PRECIP_2060_HOTDRY_JAN_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_FEB_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_MAR_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_APR_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_MAY_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_JUN_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_JUL_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_AUG_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_SEP_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_OCT_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_NOV_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_DEC_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_ANNUAL_CHANG'

    ,'PRECIP_2060_CENTRAL_JAN_CHANGEP'
    ,'PRECIP_2060_CENTRAL_FEB_CHANGEP'
    ,'PRECIP_2060_CENTRAL_MAR_CHANGEP'
    ,'PRECIP_2060_CENTRAL_APR_CHANGEP'
    ,'PRECIP_2060_CENTRAL_MAY_CHANGEP'
    ,'PRECIP_2060_CENTRAL_JUN_CHANGEP'
    ,'PRECIP_2060_CENTRAL_JUL_CHANGEP'
    ,'PRECIP_2060_CENTRAL_AUG_CHANGEP'
    ,'PRECIP_2060_CENTRAL_SEP_CHANGEP'
    ,'PRECIP_2060_CENTRAL_OCT_CHANGEP'
    ,'PRECIP_2060_CENTRAL_NOV_CHANGEP'
    ,'PRECIP_2060_CENTRAL_DEC_CHANGEP'
    ,'PRECIP_2060_CENTRAL_ANNUAL_CHAN'

    ,'PRECIP_2060_WETWARM_JAN_CHANGEP'
    ,'PRECIP_2060_WETWARM_FEB_CHANGEP'
    ,'PRECIP_2060_WETWARM_MAR_CHANGEP'
    ,'PRECIP_2060_WETWARM_APR_CHANGEP'
    ,'PRECIP_2060_WETWARM_MAY_CHANGEP'
    ,'PRECIP_2060_WETWARM_JUN_CHANGEP'
    ,'PRECIP_2060_WETWARM_JUL_CHANGEP'
    ,'PRECIP_2060_WETWARM_AUG_CHANGEP'
    ,'PRECIP_2060_WETWARM_SEP_CHANGEP'
    ,'PRECIP_2060_WETWARM_OCT_CHANGEP'
    ,'PRECIP_2060_WETWARM_NOV_CHANGEP'
    ,'PRECIP_2060_WETWARM_DEC_CHANGEP'
    ,'PRECIP_2060_WETWARM_ANNUAL_CHAN'

    ,'STORM5YR_2035_STORMY_ANNUAL_CHA'
    ,'STORM10YR_2035_STORMY_ANNUAL_CH'
    ,'STORM15YR_2035_STORMY_ANNUAL_CH'
    ,'STORM30YR_2035_STORMY_ANNUAL_CH'
    ,'STORM50YR_2035_STORMY_ANNUAL_CH'
    ,'STORM100YR_2035_STORMY_ANNUAL_C'
    
    ,'STORM5YR_2060_STORMY_ANNUAL_CHA'
    ,'STORM10YR_2060_STORMY_ANNUAL_CH'
    ,'STORM15YR_2060_STORMY_ANNUAL_CH'
    ,'STORM30YR_2060_STORMY_ANNUAL_CH'
    ,'STORM50YR_2060_STORMY_ANNUAL_CH'
    ,'STORM100YR_2060_STORMY_ANNUAL_C'
    
    ,'STORM5YR_2035_LESSSTORMY_ANNUAL'
    ,'STORM10YR_2035_LESSSTORMY_ANNUA'
    ,'STORM15YR_2035_LESSSTORMY_ANNUA'
    ,'STORM30YR_2035_LESSSTORMY_ANNUA'
    ,'STORM50YR_2035_LESSSTORMY_ANNUA'
    ,'STORM100YR_2035_LESSSTORMY_ANNU'
    
    ,'STORM5YR_2060_LESSSTORMY_ANNUAL'
    ,'STORM10YR_2060_LESSSTORMY_ANNUA'
    ,'STORM15YR_2060_LESSSTORMY_ANNUA'
    ,'STORM30YR_2060_LESSSTORMY_ANNUA'
    ,'STORM50YR_2060_LESSSTORMY_ANNUA'
    ,'STORM100YR_2060_LESSSTORMY_ANNU'
];

for fld in flds:
    
    tmp1 = arcpy.env.scratchFolder + os.sep + 'tmp1.tif';
    if arcpy.Exists(tmp1):
        arcpy.Delete_management(tmp1);
    
    print("  calculating inverse distance weighting for " + str(fld));
    searchNeighbourhood = arcpy.SearchNeighborhoodStandard(
         majorSemiaxis = idw_search_radius_val
        ,minorSemiaxis = idw_search_radius_val
    );
    arcpy.ga.IDW(
         in_features         = in_file
        ,z_field             = fld
        ,out_raster          = tmp1
        ,cell_size           = 0.5
        ,power               = idw_power
        ,search_neighborhood = searchNeighbourhood
    );
    
    tmp2 = 'memory/tmp2';
    if arcpy.Exists(tmp2):
        arcpy.Delete_management(tmp2);
        
    print("  converting raster values to points");
    arcpy.conversion.RasterToPoint(
         in_raster          = tmp1
        ,out_point_features = tmp2
        ,raster_field       = 'VALUE'
    );
    
    tmp3 = 'memory/tmp3';
    if arcpy.Exists(tmp3):
        arcpy.Delete_management(tmp3);

    print("  joining point value back into grid set");
    fieldmappings = arcpy.FieldMappings();
    fieldmappings.addTable(out_file);
    field_map = arcpy.FieldMap();
    field_map.addInputField(tmp2,'grid_code');
    fieldmappings.addFieldMap(field_map);

    arcpy.analysis.SpatialJoin(
         target_features   = out_file
        ,join_features     = tmp2
        ,out_feature_class = tmp3
        ,join_operation    = 'JOIN_ONE_TO_ONE'
        ,join_type         = 'KEEP_ALL'
        ,field_mapping     = fieldmappings
        ,match_option      = 'CONTAINS'
    );

    print("  storing value in proper field")
    arcpy.management.CalculateField(
         in_table   = tmp3
        ,field      = fld
        ,expression = '!grid_code!'
        ,expression_type = 'PYTHON3'
    );

    arcpy.management.DeleteField(
         in_table   = tmp3
        ,drop_field = [
             'grid_code'
            ,'join_count'
            ,'TARGET_FID'
        ]
    );
    
    arcpy.management.Delete(
         in_data = out_file
    );
    
    arcpy.conversion.FeatureClassToFeatureClass(
         in_features = tmp3
        ,out_path    = os.path.dirname(out_file)
        ,out_name    = os.path.basename(out_file)
    );

arcpy.management.AddField(
     in_table     = out_file
    ,field_name   = 'is_interpolated'
    ,field_type   = 'TEXT'
    ,field_length = 1
    ,field_alias  = 'Is Interpolated'
);

arcpy.management.CalculateField(
     in_table        = out_file
    ,field           = 'is_interpolated'
    ,expression      = '"Y"'
    ,expression_type = 'PYTHON3'
);

grid_interpolated_cnt  = arcpy.GetCount_management(out_file)[0];
print("  Interpolated Grid Count : " + str(grid_interpolated_cnt));


  making working version of interpolated grids
  calculating inverse distance weighting for TEMP_2035_HOTDRY_JAN_CHANGEF
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for TEMP_2035_HOTDRY_FEB_CHANGEF
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for TEMP_2035_HOTDRY_MAR_CHANGEF
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for TEMP_2035_HOTDRY_APR_CHANGEF
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for TEMP_2035_HOTDRY_MAY_CHANGEF
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance we

  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for TEMP_2060_HOTDRY_JUL_CHANGEF
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for TEMP_2060_HOTDRY_AUG_CHANGEF
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for TEMP_2060_HOTDRY_SEP_CHANGEF
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for TEMP_2060_HOTDRY_OCT_CHANGEF
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for TEMP_2060_HOTDRY_NOV_CHANGEF
  converting raster values to points
  joining point value back into grid set
  

  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for PRECIP_2035_HOTDRY_DEC_CHANGEPC
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for PRECIP_2035_HOTDRY_ANNUAL_CHANG
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for PRECIP_2035_CENTRAL_JAN_CHANGEP
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for PRECIP_2035_CENTRAL_FEB_CHANGEP
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for PRECIP_2035_CENTRAL_MAR_CHANGEP
  converting raster values to points
  joining point value back into grid set
  storing value in prope

  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for PRECIP_2060_CENTRAL_APR_CHANGEP
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for PRECIP_2060_CENTRAL_MAY_CHANGEP
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for PRECIP_2060_CENTRAL_JUN_CHANGEP
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for PRECIP_2060_CENTRAL_JUL_CHANGEP
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for PRECIP_2060_CENTRAL_AUG_CHANGEP
  converting raster values to points
  joining point value back into grid set
  storing value in prope

  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for STORM30YR_2060_LESSSTORMY_ANNUA
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for STORM50YR_2060_LESSSTORMY_ANNUA
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  calculating inverse distance weighting for STORM100YR_2060_LESSSTORMY_ANNU
  converting raster values to points
  joining point value back into grid set
  storing value in proper field
  Interpolated Grid Count : 1404
Wall time: 19min 53s


### 50.040: Swap in the interpolated grid values into the grids with data layer

In [21]:
%%time

in_file   = wrkfgdb + os.sep + 'CRWU_CREAT_Grid_Projections_DATA';
in_file2  = wrkfgdb + os.sep + 'CRWU_CREAT_Grid_Projections_INTERP_WRK'
out_file  = rezfgdb + os.sep + 'CRWU_CREAT_Grid_Projections_DATA_INTERP';

if arcpy.Exists(out_file):
    arcpy.Delete_management(out_file);
    
print("  make new copy of grids with data");
arcpy.conversion.FeatureClassToFeatureClass(
     in_features = in_file
    ,out_path    = os.path.dirname(out_file)
    ,out_name    = os.path.basename(out_file)
);

arcpy.management.AddField(
     in_table     = out_file
    ,field_name   = 'is_interpolated'
    ,field_type   = 'TEXT'
    ,field_length = 1
    ,field_alias  = 'Is Interpolated'
);

arcpy.management.CalculateField(
     in_table        = out_file
    ,field           = 'is_interpolated'
    ,expression      = '"N"'
    ,expression_type = 'PYTHON3'
);

print("  append in interpolated grids into new copy")
arcpy.management.Append(
     inputs  = in_file2
    ,target  = out_file
    ,schema_type = 'NO_TEST'
);

data_interpolated_cnt  = arcpy.GetCount_management(out_file)[0];
print("  Interpolated Grid Count : " + str(data_interpolated_cnt));

  make new copy of grids with data
  append in interpolated grids into new copy
  Interpolated Grid Count : 6332
Wall time: 6.36 s
